In [1]:
import pandas as pd
import numpy as np 
import os 
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings('ignore')

In [2]:
rev = pd.read_csv('../madrid/reviews_detailed/comments_short.csv')

In [3]:
rev = rev.dropna()

In [4]:
rev_en = rev[rev['idiomas'] == 'en']

In [5]:
rev_en.shape

(350297, 7)

In [6]:
rev_en.date=pd.to_datetime(rev_en.date, format='%Y-%m-%d')

In [7]:
sia = SentimentIntensityAnalyzer()

# Obtener el primer comentario de la columna "comments"
primer_comentario = rev_en["comments_short"].iloc[96]

# Analizar el sentimiento del comentario utilizando VADER
sentimiento = sia.polarity_scores(primer_comentario)

# Imprimir el resultado
print(sentimiento)

{'neg': 0.029, 'neu': 0.505, 'pos': 0.466, 'compound': 0.9908}


In [8]:
import os.path

# Dividir los datos en paquetes de 1000 filas
tamano_paquete = 1000
num_paquete = 0
for inicio in range(0, len(rev_en), tamano_paquete):
    fin = inicio + tamano_paquete
    paquete = rev_en.iloc[inicio:fin]

    # Analizar el sentimiento de cada comentario en el paquete
    sia = SentimentIntensityAnalyzer()
    paquete['sentimiento'] = paquete['comments_short'].apply(lambda comentario: sia.polarity_scores(comentario))

    # Guardar el paquete en un archivo separado
    nombre_archivo = f'paquete_{num_paquete}.csv'
    if not os.path.exists(nombre_archivo):
        paquete.to_csv(nombre_archivo, index=False)
    else:
        paquete.to_csv(nombre_archivo, mode='a', header=False, index=False)
    num_paquete += 1

In [11]:
import pandas as pd
import glob

# Ruta donde se encuentran los archivos CSV
path = r'C:/Users/Pedro Vidales/Desktop/Proyecto-Final_/main/vader/*.csv'

# Lee todos los archivos CSV y crea un dataframe para cada uno
all_files = glob.glob(path)
df_list = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(df)

# Concatena los dataframes en uno solo
rev_paquetes3 = pd.concat(df_list, axis=0, ignore_index=True)

# Muestra el resultado
rev_paquetes3.head()

,listing_id,id,date,reviewer_id,comments,idiomas,comments_short,sentimiento
0,6369,29428,2010-03-14,84790,simon and arturo have the ultimate location in...,en,arturo ultimate location step bus line want vi...,"{'neg': 0.0, 'neu': 0.623, 'pos': 0.377, 'comp..."
1,6369,31018,2010-03-23,84338,myself and kristy originally planned on stayin...,en,plan stay arturo week plan change arturo open ...,"{'neg': 0.025, 'neu': 0.357, 'pos': 0.618, 'co..."
2,6369,34694,2010-04-10,98655,we had a great time at arturo and simon's ! a ...,en,great time arturo cosy apartment wonderful ter...,"{'neg': 0.0, 'neu': 0.314, 'pos': 0.686, 'comp..."
3,6369,37146,2010-04-21,109871,i very much enjoyed the stay. \r\nit's a wond...,en,enjoy stay wonderful room bath great apartment...,"{'neg': 0.0, 'neu': 0.34, 'pos': 0.66, 'compou..."
4,6369,38168,2010-04-26,98901,arturo and simon are polite and friendly hosts...,en,polite friendly host provide pleasant convenie...,"{'neg': 0.0, 'neu': 0.48, 'pos': 0.52, 'compou..."


In [12]:
import json
def parse_sentimientos(sentimiento):
    return json.loads(sentimiento.replace("'", "\""))

# Convertimos los datos en formato JSON a un diccionario de Python
rev_paquetes3['sentimiento_dict'] = rev_paquetes3['sentimiento'].apply(parse_sentimientos)

In [13]:
# Creamos cuatro columnas nuevas y extraemos los valores correspondientes a cada clave en cada fila
rev_paquetes3['neg'] = rev_paquetes3['sentimiento_dict'].apply(lambda x: x['neg'])
rev_paquetes3['neu'] = rev_paquetes3['sentimiento_dict'].apply(lambda x: x['neu'])
rev_paquetes3['pos'] = rev_paquetes3['sentimiento_dict'].apply(lambda x: x['pos'])
rev_paquetes3['compound'] = rev_paquetes3['sentimiento_dict'].apply(lambda x: x['compound'])


In [14]:
rev = rev_paquetes3.drop(['sentimiento', 'sentimiento_dict'], axis=1)